# <font size="10">Custom entity recognition </font>
## Model environment setup

This notebook contains test code to train the implemented model in the generated training data outputted from
the [ner-train notebook located here](./ner-train-note.ipynb).

For simple loop model training (old), go [here](## Run training - using simple training loop from blank -- Old)

## Setup

In [1]:
!pip install -U spacy
!pip install spacy[lookups]


     |████████████████████████████████| 10.0MB 2.7MB/s 
     |████████████████████████████████| 2.1MB 48.2MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 93.8MB 56kB/s 
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.3.2-py2.py3-none-any.whl size=93807573 sha256=2a1ad5f153362ddfe985f4eb79b0e55ea91e0c9573ad12f1c5978fa8d6372c8d
  Stored in directory: /root/.cache/pip/wheels/5b/f4/d0/bf720a06127c95d9be2a81d197a3f1998ee5fc63410944e28f
Successfully built spacy-lookups-data


In [22]:
!python -m spacy download en

!python -m spacy download en_core_web_md
!python -m spacy validate

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
✔ Loaded compatibility table

====================== Installed models (spaCy v2.3.2) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.3.1   ✔
package   en-core-web-md   en_core_web_md   2.3.1   ✔
link      en               en_core_web_sm   2.3.1   ✔



In [16]:
import pandas as pd
import random
import warnings
import json
import ast
import datetime as dt
from pathlib import Path
import os
import glob
from __future__ import unicode_literals, print_function

##SpaCy

import en_core_web_sm
import en_core_web_md

import spacy
from spacy import displacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.pipeline import Sentencizer
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB
from spacy.util import minibatch, compounding
from spacy.util import decaying
from spacy.pipeline import Tagger
from spacy.pipeline import DependencyParser
from thinc.neural.optimizers import Adam



# For Colab

In [4]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# For Local

In [5]:
'''train_path = r'./train/'

csvs = glob.glob(train_path + "/*.csv")


print('Filepath is :',(csvs))

train_list = []

for filename in csvs:
    df = pd.read_csv(filename, index_col=None, header=0)
    train_list.append(df)'''


'train_path = r\'./train/\'\n\ncsvs = glob.glob(train_path + "/*.csv")\n\n\nprint(\'Filepath is :\',(csvs))\n\ntrain_list = []\n\nfor filename in csvs:\n    df = pd.read_csv(filename, index_col=None, header=0)\n    train_list.append(df)'

## Import training data

lets import the training data we generated:

In [58]:
train_path = r'/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/'

csvs = glob.glob(train_path + "/*.csv")


print('Filepath is :',(csvs))

train_list = []

for filename in csvs:
    df = pd.read_csv(filename, index_col=None, header=0)
    train_list.append(df)


Filepath is : ['/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset0.csv', '/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset1.csv', '/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset2.csv', '/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset3.csv']


In [59]:
DATA = pd.concat(train_list, axis=0, ignore_index=True)
DATA = DATA.drop_duplicates()
DATA[0:10]
print(len(DATA))

3249


In [60]:
#convert to list for model intake
TRAIN_DATA = DATA.values.tolist()

#for element in index 1 convert string (Entity position) to dictionary to be able to read by the model function
for position in TRAIN_DATA:
    position[1]=ast.literal_eval(position[1])
    
#Check our input list
TRAIN_DATA[0:10]

[['With increasing severe weather events and disasters \ntriggering greater numbers of costly power outages, there is a growing interest in generators for \nreliable backup power.',
  {'entities': [(161, 173, 'SUSTECH')]}],
 ['$3,153 \n\nNPC of Backup Power per Unit ($/kW)',
  {'entities': [(16, 28, 'SUSTECH')]}],
 ['Annualized Costs and Benefits per Unit of Backup Power ($/kW-year).',
  {'entities': [(42, 54, 'SUSTECH')]}],
 ['Many businesses without backup are \nconsidering installing generators, and facilities such as hospitals and airports—which are \nrequired to have backup power—are considering redundant systems for added resilience against \ngrid outages.',
  {'entities': [(145, 157, 'SUSTECH')]}],
 ['Revenues from additional services can \nsignificantly reduce the life-cycle costs of a backup power system.',
  {'entities': [(86, 98, 'SUSTECH')]}],
 ['Generators that provide \ngrid services in addition to backup power are often run more frequently, which generally \ncoincides wi

## Run a test before training
### Test existing default spacy model 

In [9]:
nlp = en_core_web_md.load()
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7faf0d51be10>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7faf0efb08e8>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7faf0e383e28>)]

In [10]:
doc = nlp('Here is a green roof on this house. A green roof is good.')
displacy.render(doc, style="ent")
# verified green roof does not match an entity in the NER

/usr/local/lib/python3.6/dist-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


'<div class="entities" style="line-height: 2.5; direction: ltr">Here is a green roof on this house. A green roof is good.</div>'

## TRAINING THE MODEL
## Train model setup

### Define compounding batch size

In [11]:
def get_batches(train_data, model_type):
    max_batch_sizes = {"tagger": 32, "parser": 16, "ner": 16, "textcat": 64}
    max_batch_size = max_batch_sizes[model_type]
    if len(train_data) < 1000:
        max_batch_size /= 2
    if len(train_data) < 500:
        max_batch_size /= 2
    batch_size = compounding(1, max_batch_size, 1.001)
    batches = minibatch(train_data, size=batch_size)
    return batches

## Define custom Adam optimizer





In [12]:

def custom_optimizer(optimizer, learn_rate=0.001, beta1=0.9, beta2=0.999, eps=1e-8, L2=1e-6, max_grad_norm=1.0):
    """
    Function to customizer spaCy default optimizer
    """
    
    optimizer.learn_rate = learn_rate
    optimizer.beta1 = beta1
    optimizer.beta2 = beta2
    optimizer.eps = eps
    optimizer.L2 = L2
    optimizer.max_grad_norm = max_grad_norm
    
    return optimizer

## Training Loop

In [116]:
#New training model loop to either accept existing model, if not model is not defined then create a blank nlp model using english vocab

def train_model(train_data = TRAIN_DATA, model=en_core_web_md, output_dir=None, n_iter=100,
        learn_rate=0.001, 
        beta1=0.9, 
        beta2=0.999, 
        eps=1e-8, 
        L2=1e-6, 
        max_grad_norm=1.0):
   
   
    random.seed(0)
    
    if model is not None:
        nlp = model.load() #load existing spacy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")
        #lang = "en"
        #cls = spacy.util.get_lang_class(lang)   # 1. Get Language instance, e.g. English()
        #nlp = cls() 
        print("Created blank 'en' model")
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise,get it, so we can add labels to it
    else:
        nlp.remove_pipe("ner")
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
        ner = nlp.get_pipe("ner")

    #ner.add_label(LABEL)  # add new entity label to entity recognizer
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            #print(ent[2])

    # Adding extraneous labels shouldn't mess anything up
    ner.add_label("VEGETABLE")
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    # only train NER
    
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        
        warnings.filterwarnings("once", category=UserWarning, module='spacy')
        if model is None:
            optimizer = nlp.begin_training(component_cfg={"ner": {"conv_window": 3}})
            optimizer=custom_optimizer(optimizer, learn_rate=learn_rate)

        else:
            optimizer = nlp.begin_training(component_cfg={"ner": {"conv_window": 3}})
            optimizer=custom_optimizer(optimizer, learn_rate=learn_rate)


        # Define decaying dropout
        dropout = decaying(0.6, 0.2, 1e-4)

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        for itn in range(n_iter):
            random.shuffle(train_data)
            #batches = get_batches(TRAIN_DATA, 'ner') resulted in poor loss
            batches = minibatch(train_data,  size=compounding(4.0, 32.0, 1.001))
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, 
                           annotations,
                           sgd=optimizer, 
                           drop = next(dropout), 
                           losses=losses)
            print(f"Losses at iteration {itn} - {dt.datetime.now()} {losses}")
    
    print('Model training completed')
    return nlp

In [117]:
model_params = {
    'model': en_core_web_md,
    'iterations': 40,
    'train_data': TRAIN_DATA,
    'dropout': decaying(0.6, 0.2, 1e-4),
    'learn_rate':0.001, 
    'beta1': 0.9, 
    'beta2': 0.999, 
    'eps': 1e-8, 
    'L2': 1e-6, 
    'max_grad_norm':1.0
}

#begin training customizable options 
'''
"beam_width":1,
"beam_density":0.0,
"beam_update_prob":1.0,
"cnn_maxout_pieces":3,
"nr_feature_tokens":6,
"nr_class":10,
"hidden_depth":1,
"token_vector_width":96,
"hidden_width":64,
"maxout_pieces":2,
"pretrained_vectors":null,
"bilstm_depth":0,
"self_attn_depth":0,
"conv_depth":4,
"conv_window":3,
"embed_size":2000
'''

'\n"beam_width":1,\n"beam_density":0.0,\n"beam_update_prob":1.0,\n"cnn_maxout_pieces":3,\n"nr_feature_tokens":6,\n"nr_class":10,\n"hidden_depth":1,\n"token_vector_width":96,\n"hidden_width":64,\n"maxout_pieces":2,\n"pretrained_vectors":null,\n"bilstm_depth":0,\n"self_attn_depth":0,\n"conv_depth":4,\n"conv_window":3,\n"embed_size":2000\n'

In [118]:
nlp = train_model()

Loaded model '<module 'en_core_web_md' from '/usr/local/lib/python3.6/dist-packages/en_core_web_md/__init__.py'>'
Losses at iteration 0 - 2020-10-29 23:32:24.821244 {'ner': 1631.2920373630238}
Losses at iteration 1 - 2020-10-29 23:32:59.759726 {'ner': 112.09180070007179}
Losses at iteration 2 - 2020-10-29 23:33:36.597123 {'ner': 87.3442861223907}
Losses at iteration 3 - 2020-10-29 23:34:15.186907 {'ner': 63.975103423017394}
Losses at iteration 4 - 2020-10-29 23:34:54.072759 {'ner': 53.39578296605587}
Losses at iteration 5 - 2020-10-29 23:35:33.118571 {'ner': 78.64864063497745}
Losses at iteration 6 - 2020-10-29 23:36:12.031120 {'ner': 67.82984833510227}
Losses at iteration 7 - 2020-10-29 23:36:54.372536 {'ner': 66.58055499147947}
Losses at iteration 8 - 2020-10-29 23:37:32.370393 {'ner': 67.53874912118522}
Losses at iteration 9 - 2020-10-29 23:38:12.933429 {'ner': 52.31454281694933}
Losses at iteration 10 - 2020-10-29 23:38:50.512528 {'ner': 51.872664481498475}
Losses at iteration 11 -

## Test the trained model

In [119]:
nlp.pipeline 

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7faeeb44fe10>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7faef5891948>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7faefba214c8>)]

In [120]:
doc = nlp('Here is a green roof on this house. A green roof is good. water piping, I have alot of battery packs')


In [121]:
displacy.render(doc, style="ent")

'<div class="entities" style="line-height: 2.5; direction: ltr">Here is a \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    green roof\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">SUSTECH</span>\n</mark>\n on this house. A green \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    roof is\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">VEGETABLE</span>\n</mark>\n good. water piping, I have alot of battery packs</div>'

## Save model for testing

In [122]:
output_dir = r'/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/model'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/model


In [ ]:
'''output_dir = r'./model'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)'''

## Loading and testing the saved model

In [123]:
x = ['i am a green roof']
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

for text in x:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from /content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/model
Entities [('green roof', 'SUSTECH')]
Tokens [('i', '', 2), ('am', '', 2), ('a', '', 2), ('green', 'SUSTECH', 3), ('roof', 'SUSTECH', 1)]


## For Local

In [ ]:
'''output_dir = r'./model'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)'''

In [ ]:
'''output_dir = r'./model'
x = ['rainwater harvesting is great']
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

for text in x:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])'''